# Eigenvalues of the dielectric matrix

We compute a few eigenvalues of the dielectric matrix ($q=0$, $ω=0$) iteratively.

In [1]:
using DFTK
using Plots
using KrylovKit
using Printf

# Calculation parameters
kgrid = [1, 1, 1]
Ecut = 5

# Silicon lattice
a = 10.26
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

# Compute the dielectric operator without symmetries
model = model_LDA(lattice, atoms, positions, symmetries=false)
basis = PlaneWaveBasis(model; Ecut, kgrid)
scfres = self_consistent_field(basis, tol=1e-14);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.232324719300                   -0.50    6.0
  2   -7.250072147357       -1.75       -1.41    1.0
  3   -7.251321318464       -2.90       -2.14    4.0
  4   -7.251313006356   +   -5.08       -2.36    3.0
  5   -7.251334479673       -4.67       -2.82    2.0
  6   -7.251337263364       -5.56       -3.06    2.0
  7   -7.251338776972       -5.82       -3.99    1.0
  8   -7.251338795204       -7.74       -4.29    3.0
  9   -7.251338798541       -8.48       -4.99    2.0
 10   -7.251338798694       -9.81       -5.54    3.0
 11   -7.251338798702      -11.10       -5.86    3.0
 12   -7.251338798704      -11.69       -6.37    2.0
 13   -7.251338798705      -12.85       -6.82    2.0
 14   -7.251338798705      -13.60       -7.27    2.0
 15   -7.251338798705   +    -Inf       -7.77    3.0


Applying $ε^† ≔ (1- χ_0 K)$ …

In [2]:
function eps_fun(δρ)
    δV = apply_kernel(basis, δρ; ρ=scfres.ρ)
    χ0δV = apply_χ0(scfres, δV)
    δρ - χ0δV
end;

… eagerly diagonalizes the subspace matrix at each iteration

In [3]:
eigsolve(eps_fun, randn(size(scfres.ρ)), 5, :LM; eager=true, verbosity=3);

[ Info: Arnoldi iteration step 1: normres = 0.08775779935084296
[ Info: Arnoldi iteration step 2: normres = 0.5826214842892241
[ Info: Arnoldi iteration step 3: normres = 0.5964640343528758
[ Info: Arnoldi iteration step 4: normres = 0.3023987998321548
[ Info: Arnoldi iteration step 5: normres = 0.21364992489943466
[ Info: Arnoldi schursolve in iter 1, krylovdim = 5: 0 values converged, normres = (5.09e-03, 4.39e-02, 1.38e-01, 1.55e-01, 2.54e-02)
[ Info: Arnoldi iteration step 6: normres = 0.6224212912359095
[ Info: Arnoldi schursolve in iter 1, krylovdim = 6: 0 values converged, normres = (2.31e-03, 8.17e-02, 5.99e-01, 1.03e-01, 9.61e-02)
[ Info: Arnoldi iteration step 7: normres = 0.07993087159799198
[ Info: Arnoldi schursolve in iter 1, krylovdim = 7: 0 values converged, normres = (1.21e-04, 4.31e-02, 1.28e-02, 2.34e-02, 5.25e-02)
[ Info: Arnoldi iteration step 8: normres = 0.10639045332575037
[ Info: Arnoldi schursolve in iter 1, krylovdim = 8: 0 values converged, normres = (5.55e-